# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port elizabeth,-33.9180,25.5701,15.61,88,75,6.69,ZA,1691517980
1,1,san patricio,28.0170,-97.5169,37.40,35,2,6.38,US,1691517981
2,2,ilulissat,69.2167,-51.1000,10.01,57,40,2.06,GL,1691517981
3,3,olonkinbyen,70.9221,-8.7187,4.71,91,100,6.19,SJ,1691517981
4,4,jamestown,42.0970,-79.2353,23.62,64,75,8.75,US,1691517827


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 500,
    frame_height = 300,
    size = "Humidity",
    #scale = 0.01,
    color = "City"
)


# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
idealcities_df = city_data_df.loc[(city_data_df["Max Temp"]>=23)&
                                  (city_data_df["Max Temp"]<=25)&
                                  (city_data_df["Cloudiness"]<=30)&
                                  (city_data_df["Humidity"]>=30)&
                                  (city_data_df["Wind Speed"]<=5),:]

# Drop any rows with null values
idealcities_df.dropna()

# Display sample data
idealcities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
56,56,pangai,-19.8000,-174.3500,24.45,89,12,4.54,TO,1691517991
77,77,zapolyarnyy,69.4154,30.8136,24.60,75,0,3.02,RU,1691517995
134,134,yatou,37.1500,122.3833,23.46,93,5,3.01,CN,1691518004
154,154,fort frances,48.5999,-93.4003,24.53,65,20,3.60,CA,1691518007
156,156,tromso,69.6496,18.9570,24.01,60,0,4.12,NO,1691518008


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(idealcities_df[["City","Country","Lat","Lng","Humidity"]])

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
56,pangai,TO,-19.8000,-174.3500,89,
77,zapolyarnyy,RU,69.4154,30.8136,75,
134,yatou,CN,37.1500,122.3833,93,
154,fort frances,CA,48.5999,-93.4003,65,
156,tromso,NO,69.6496,18.9570,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row[3]
    lat = row[2]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
pangai - nearest hotel: Billy's Place
zapolyarnyy - nearest hotel: Отель Печенга
yatou - nearest hotel: No hotel found
fort frances - nearest hotel: The Sleepy Owl
tromso - nearest hotel: Clarion Collection Hotel Aurora
alofi - nearest hotel: Matavai Resort
creston - nearest hotel: No hotel found
n'dalatando - nearest hotel: Hotel Terminus
manado - nearest hotel: Swiss Belhotel
mugia - nearest hotel: Solpor do Coído
miahuatlan - nearest hotel: Hotel Global express
orbetello - nearest hotel: B&B La Pineta
kirkland lake - nearest hotel: No hotel found
rodolfo sanchez taboada - nearest hotel: Estero Beach Resort
ouani - nearest hotel: Al Amal


,City,Country,Lat,Lng,Humidity,Hotel Name
56,pangai,TO,-19.8000,-174.3500,89,Billy's Place
77,zapolyarnyy,RU,69.4154,30.8136,75,Отель Печенга
134,yatou,CN,37.1500,122.3833,93,No hotel found
154,fort frances,CA,48.5999,-93.4003,65,The Sleepy Owl
156,tromso,NO,69.6496,18.9570,60,Clarion Collection Hotel Aurora
198,alofi,NU,-19.0595,-169.9187,87,Matavai Resort
245,creston,US,41.0586,-94.3614,78,No hotel found
301,n'dalatando,AO,-9.2978,14.9116,42,Hotel Terminus
305,manado,ID,1.4870,124.8455,83,Swiss Belhotel
311,mugia,ES,43.1041,-9.2179,73,Solpor do Coído


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Drop cities with no hotel near
hotel_df_clean = hotel_df.loc[(hotel_df["Hotel Name"]!="No hotel found")]

# Configure the map plot
map_with_hotels = hotel_df_clean.hvplot.points("Lng","Lat",geo=True,tiles="EsriNatGeo",    
                                         frame_width = 500,
                                         frame_height = 300,
                                         size = "Humidity",
                                         color = "City",
                                         hover_cols = ["Country","Hotel Name"])

# Display the map
map_with_hotels

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)